In [1]:
# https://www.chosun.com/culture-life/culture_general/?page=2 식

section_dict={"economy/economy_general":"경제","politics/politics_general":"정치","national/national_general":"사회","international/international_general":"국제","sports/sports_general":"스포츠","entertainments/enter_general":"연예","culture-life/culture_general":"문화"}

In [2]:
sections = list(section_dict.keys())

In [3]:
sections

['economy/economy_general',
 'politics/politics_general',
 'national/national_general',
 'international/international_general',
 'sports/sports_general',
 'entertainments/enter_general',
 'culture-life/culture_general']

In [17]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import numpy as np
import pandas as pd
from selenium import webdriver

In [5]:
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [6]:
def chosun_make_link_list(section_name,pageidx):
    url = "https://www.chosun.com/{}/?page={}".format(section_name,pageidx)
    
    driver.implicitly_wait(0.3)
    driver.get(url)
    html = driver.page_source
    html = BeautifulSoup(html, 'html.parser')
    news_list=html.findAll("a",attrs={'class': 'text__link'})
    news_link_list=[]
    cnt=0
    for i in news_list:
        news_link_list.append(str(i.attrs['href']))
        if cnt>=18:
            break
        cnt+=1
    return news_link_list

In [7]:
def chosun_select_section_auto(sectionlist,startpage,stoppage):
    autolinklist=[]
    for section in sectionlist:
        for i in range(startpage,stoppage):
            temp=chosun_make_link_list(section,i)
            if not temp:
                continue
            autolinklist+=temp
        time.sleep(abs((np.random.randn())*0.1))

    return autolinklist

In [18]:
def chosun_link_to_df(link_list):
    root_url="https://www.chosun.com"
    data=[]
    

    for link in link_list:
        url = root_url+link
        driver.implicitly_wait(0.3)
        driver.get(url)
        html = driver.page_source

        pagehtml = BeautifulSoup(html, 'html.parser')
        
        
        
        pagetitle=pagehtml.h1
        pagecontent=pagehtml.find("section",attrs={'class': 'article-body'})
        pagedate=pagehtml.find("div",attrs={'class': 'article-dateline'})
        
        temp=url.split(".chosun.com/")
        temp=temp[-1].split("/2")
        pagesection = temp[0]
        
        if temp[0] in sections:
            pagesection = section_dict[temp[0]]
        
        if pagetitle and pagecontent and pagedate and pagesection:
            pagetitle=pagetitle.text
            pagecontent=pagecontent.text
            pagedate=pagedate.text.replace("입력","")
            
        data.append([pagetitle,pagecontent,pagedate,pagesection,url])
        
        time.sleep(abs((np.random.randn())*0.05))
        
    return pd.DataFrame(data, columns=['title','content','date','section','url'])

In [9]:
#startdate를 읽어온 csv 파일에서 가장 최신 날짜로 자동으로 잡도록...하고 싶었지만 생각해보니 애초에 날짜 저장을 안 했다
def auto_save(startpage,pages=3):
    url_df = fileread("chosun_news_url.csv")
    c=chosun_select_section_auto(sections,startpage,startpage+pages)
    url_df_temp = pd.DataFrame(c,columns=['news_url'])
    url_df = pd.concat([url_df,url_df_temp])
    filesave(url_df,"chosun_news_url.csv")
    print(startpage+pages,"까지 저장")

In [103]:
# # 처음 실행시 데이터프레임 비우는 코드
# # 두번째 이후에 실행하지 않도록 주의 (파일 날아감)
# url_df=pd.DataFrame(columns=['news_url'])
# filesave(url_df,"chosun_news_url.csv")

<ipython-input-103-4957f6b60c99>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')


In [107]:
# 기사의 url을 가져오는 코드
# 가져오면 chosun_news_url.csv에 저장됨.
# chosun_news_url.csv는 첨부할 테니 이 셀은 돌리지 말 것.

driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')
count=181

for i in range(60):
    auto_save(count)
    count+=3

184 까지 저장
187 까지 저장
190 까지 저장
193 까지 저장
196 까지 저장
199 까지 저장
202 까지 저장
205 까지 저장
208 까지 저장


KeyboardInterrupt: 

In [10]:
test=fileread("chosun_news_url.csv")

In [11]:
test

,news_url
0,/economy/economy_general/2022/10/25/L6LM4QPAZF...
1,/economy/economy_general/2022/10/25/HH3S6YI6CN...
2,/economy/economy_general/2022/10/25/IIDTJWYION...
3,/economy/economy_general/2022/10/25/RREKVAFDBJ...
4,/economy/economy_general/2022/10/25/HGFE3YHQYZ...
...,...
24825,/culture-life/culture_general/2021/04/05/CSH6G...
24826,/culture-life/2021/04/05/PHANEHPXYVAQJOZUK5KBO...
24827,/culture-life/culture_general/2021/04/04/OC7Q4...
24828,/culture-life/2021/04/03/BFLHKQCUGNF7PKUQRBSU7...


In [12]:
# 처음 실행할 경우 chosun_df를 빈 데이터프레임으로 만들기.
chosun_df = pd.DataFrame(columns=['title','content','date','section'])

In [13]:
# 기사의 url을 불러와서 리스트에 넣어줌
test=fileread("chosun_news_url.csv")
splitedlist=[test[i*10:i*10+10] for i in range(0,(len(test)//10)+1)]
print("length of for : ",len(splitedlist))

length of for :  2484


In [19]:
# 저장된 파일에서 이어서 불러오는 코드
chosun_df = fileread("chosun_dataframe.csv")

In [20]:
chosun_df

,title,content,date,section,url
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
...,...,...,...,...,...
1985,"정성호 “이재명, 부패지옥·청렴천국 달고 살아… 김용 개인비리일수도”",이재명 더불어민주당 대표는 24일 검찰이 서울 여의도 민주당사 내 민주연구원을 압수...,2022.10.25 07:17,정치,https://www.chosun.com/politics/politics_gener...
1986,"한동훈에 “화내는 모습 부적절” 지적한 김남국, 잠시 후 “이 사람아” 고성",24일 법제사법위원회 국정감사에서 화제가 된 장면은 한동훈 법무부 장관과 김의겸 더...,2022.10.25 10:25,정치,https://www.chosun.com/politics/politics_gener...
1987,“이게 6만원…먹다 남은 거 아닙니다” 속초 횟집 가격 논란,강원도 속초의 한 횟집에서 6만원 어치 참돔과 2만원 어치 개불을 포장해왔다며 네티...,2022.10.25 11:48,사회,https://www.chosun.com/national/national_gener...
1988,시정 연설·사전차담회 모두 거부한 野… 5년 전 文때는 달랐다,윤석열 대통령이 25일 서울 여의도 국회에서 2023년도 예산안 및 기금운용계획안 ...,2022.10.25 10:06,politics/assembly,https://www.chosun.com/politics/assembly/2022/...


In [ ]:
#실제 크롤링 코드. 10페이지 가져올 때마다 chosun_dataframe.csv에 저장됨
# 오류 났을 경우 어디에서 오류 났는지 보고 count를 바꿔 재실행
# 예시 : "39에서 에러 발생" 일 때 -> count = 39로 두고 재실행
driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver")
count=1508
for _ in range(5):
    try:
        for linklist in splitedlist[count:]:
            start = time.time()
            print("start")
            chosun_df_temp = chosun_link_to_df(list(linklist['news_url']))
            chosun_df = pd.concat([chosun_df,chosun_df_temp])
            filesave(chosun_df,"chosun_dataframe.csv")
            print("saved")
            count+=1

            print("소요시간 :", time.time() - start)
            print(count*10,"번째까지 저장")

            time.sleep((random.random()*0.2))
    except:
        print(count,"에서 에러 발생")
        driver.close()
        time.sleep(3)
        driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14676\3290673977.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver")


start
saved
소요시간 : 25.186458349227905
15090 번째까지 저장
start
saved
소요시간 : 17.877341508865356
15100 번째까지 저장
start
saved
소요시간 : 24.999627351760864
15110 번째까지 저장
start
saved
소요시간 : 21.135898113250732
15120 번째까지 저장
start
saved
소요시간 : 21.958678007125854
15130 번째까지 저장
start
saved
소요시간 : 27.489112615585327
15140 번째까지 저장
start
saved
소요시간 : 26.66584038734436
15150 번째까지 저장
start
saved
소요시간 : 26.332737684249878
15160 번째까지 저장
start
saved
소요시간 : 23.806476593017578
15170 번째까지 저장
start
saved
소요시간 : 27.410828351974487
15180 번째까지 저장
start
saved
소요시간 : 25.442505598068237
15190 번째까지 저장
start
saved
소요시간 : 24.37313413619995
15200 번째까지 저장
start
saved
소요시간 : 26.269855260849
15210 번째까지 저장
start
saved
소요시간 : 21.325993299484253
15220 번째까지 저장
start
saved
소요시간 : 24.861745834350586
15230 번째까지 저장
start
saved
소요시간 : 27.655099391937256
15240 번째까지 저장
start
saved
소요시간 : 24.70732045173645
15250 번째까지 저장
start
saved
소요시간 : 30.132850170135498
15260 번째까지 저장
start
saved
소요시간 : 26.29125952720642
15270 번째까지 저장
start
saved
소요시간 : 

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14676\3290673977.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver")


start
saved
소요시간 : 27.586084365844727
15910 번째까지 저장
start
saved
소요시간 : 20.86780095100403
15920 번째까지 저장
start
saved
소요시간 : 23.655811309814453
15930 번째까지 저장
start
saved
소요시간 : 26.809151649475098
15940 번째까지 저장
start
saved
소요시간 : 26.679012060165405
15950 번째까지 저장
start
saved
소요시간 : 26.800082683563232
15960 번째까지 저장
start
saved
소요시간 : 25.21640110015869
15970 번째까지 저장
start


In [142]:
driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')
count=36

for linklist in splitedlist[count:]:
    start = time.time()
    print("start")
    chosun_df_temp = chosun_link_to_df(list(linklist['news_url']))
    chosun_df = pd.concat([chosun_df,chosun_df_temp])
    filesave(chosun_df,"chosun_dataframe.csv")
    print("saved")
    count+=1

    print("소요시간 :", time.time() - start)
    print(count*10,"번째까지 저장")

    time.sleep((random.random()*0.2))

<ipython-input-142-a60af6c08eb1>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')


start


UnboundLocalError: local variable 'pagesection' referenced before assignment

In [36]:
chosun_df

,title,content,date,section,url
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
...,...,...,...,...,...
5,마리우폴 최후 항전지 아조우스탈서 우크라·러군 혈투,5월 4일 도네츠크 정부가 공개한 영상에서 캡처한 사진. 우크라이나 마리우폴 아조우...,2022.05.05 23:30,국제,https://www.chosun.com/international/internati...
6,"AP “마리우폴 극장 폭격 희생자, 당초 2배인 600여명 추정”",지난 3월 16일(현지 시각) 러시아군의 공습을 받은 우크라이나 마리우폴의 극장 모...,2022.05.05 23:05,국제,https://www.chosun.com/international/internati...
7,"“손도 못대게 하는 데 발을”… 한예슬, 美 협곡 올라타고 인증샷",/한예슬 인스타그램배우 한예슬이 미국 관광지에서 부적절한 사진을 촬영해 ‘무개념 인...,2022.05.05 20:37,국제,https://www.chosun.com/international/internati...
8,이웃 노인들에 러 공습 알리려다 사망…우크라 14세 ‘영웅 소년’,/트위터우크라이나 14세 소년이 이웃 노인들에게 러시아의 공습을 알리려다 숨졌다.4...,2022.05.05 17:12,국제,https://www.chosun.com/international/internati...
